**Methods/Results**

In [3]:
import pandas as pd
import altair as alt
import numpy as np

url= "https://docs.google.com/spreadsheets/d/e/2PACX-1vROC4kgO6ctTkCjDooBh4Gc_VW7fsUeIgSiPTtcHV0FjFumQclEF8b3ThtxYAJQPyDmRN61OpR4gnpr/pub?output=csv"
pulsar_data = pd.read_csv(url, header= None, names =[
    "integrated_mean",
    "integrated_sd",
    "integrated_xs_kurtosis",
    "integrated_skewness",
    "dmsnr_mean",
    "dmsnr_sd",
    "dmsnr_xs_kurtosis",
    "dmsnr_skewness",
    "class"
],)

pulsar_data["class"]=pulsar_data["class"].replace({
    0: "not pulsar",
    1: "pulsar"
})
pulsar_data = pulsar_data.drop(columns=["dmsnr_mean", "dmsnr_sd", "dmsnr_xs_kurtosis", "dmsnr_skewness"])
pulsar_data

,integrated_mean,integrated_sd,integrated_xs_kurtosis,integrated_skewness,class
0,140.562500,55.683782,-0.234571,-0.699648,not pulsar
1,102.507812,58.882430,0.465318,-0.515088,not pulsar
2,103.015625,39.341649,0.323328,1.051164,not pulsar
3,136.750000,57.178449,-0.068415,-0.636238,not pulsar
4,88.726562,40.672225,0.600866,1.123492,not pulsar
...,...,...,...,...,...
17893,136.429688,59.847421,-0.187846,-0.738123,not pulsar
17894,122.554688,49.485605,0.127978,0.323061,not pulsar
17895,119.335938,59.935939,0.159363,-0.743025,not pulsar
17896,114.507812,53.902400,0.201161,-0.024789,not pulsar
